In [2]:
import os
import io
import time
import logging
import random
import re
import sqlite3
from datetime import datetime, timedelta, date
from typing import List, Dict, Any
import pandas as pd
import numpy as np
import gspread
from sqlalchemy import create_engine
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, HttpRequest
from googleapiclient.errors import HttpError

# SCRIPT DIRECTORY MADE DYNAMIC 
extrapolated to other sections since folders created

In [3]:
script_dir = os.getcwd()
print(script_dir)

#background_data_dir = script_dir + '\\' + 'Background Data Folder'
"""os.path.join adds the appropriate path separator (\ or /) based on the operating system"""
background_data_dir = os.path.join(script_dir, 'Background Data Folder')
if not os.path.exists(background_data_dir):
    os.makedirs(background_data_dir)

#report_dir = script_dir + '\\' + 'Reports Folder'
report_dir = os.path.join(script_dir, 'Reports Folder') 
if not os.path.exists(report_dir):
    os.makedirs(report_dir)

data_supplied_dir = os.path.join(script_dir, 'Data Supplied Folder')
if not os.path.exists(data_supplied_dir):
    os.makedirs(data_supplied_dir)

c:\Users\Nicola\repositories & enviroments\Python\Automation\Weekly Team Update Meeting


In [ ]:
def list_files_in_folder(script_dir):

    if not os.path.exists(script_dir):
        print(f"Directory not found: {script_dir}")
        return []

    files = [f for f in os.listdir(script_dir) if os.path.isfile(os.path.join(script_dir, f))]
    
    print(f"Files in {script_dir}:")
    for file in files:
        print(file)
    
    return #files

In [17]:
list_files_in_data_supplied_folder(data_supplied_dir)

Files in c:\Users\nicola\Desktop\VisualCode Workspace\Team Meeting Update\Data Supplied Folder:
KIT 3 Online Reporting Data 01 05 2025.xlsx
previous_errors_n_reponse_times_data.db


# CONNECT TO GOOGLE DRIVE

In [ ]:
def authenticate_gdrive():
    try:
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        drive_service = build('drive', 'v3', credentials=creds)
        return gc, drive_service

    except Exception as e:
        logging.error(f"Failed to authenticate with Google API: {e}")
    raise

In [ ]:
def download_file(file_id: str, output_filename: str):
    try:
        _, drive_service = authenticate_gdrive()
        request = drive_service.files().export_media(fileId=file_id,mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
    
        with io.FileIO(output_filename, 'wb') as fh:
            downloader = MediaIoBaseDownload(fh, request)
            done = False

        while not done:
            status, done = downloader.next_chunk()
            logging.info(f"Download {int(status.progress() * 100)}% complete.")

        logging.info(f"File downloaded as {output_filename}")
    
    except Exception as e:
        logging.error(f"Failed to download file: {e}")
        raise


In [18]:
def execute_with_backoff(func,*args,max_retries=5,**kwargs):
    for attempt in range(max_retries):
        try:
            result=func(*args,**kwargs)
            if isinstance(result,HttpRequest):
                result=result.execute()

            return result

        except(HttpError,gspread.exceptions.APIError) as e:
            if 'Quota exceeded' in str(e) or getattr(e,'status',0)==429:

                if attempt==max_retries-1:
                    logging.error(f"Max retries reached for {func.__name__}: {e}")
                    raise

                sleep_time=(2**attempt)+(random.randint(0,1000)/1000)
                logging.warning(f"Quota exceeded, retrying in {sleep_time:.2f} seconds...")
                time.sleep(sleep_time)

            else:
                raise

        except Exception as e:
            logging.error(f"Unexpected error in {func.__name__}: {e}")
            raise

In [19]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

SCOPES = ['https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive']

# Configure logging
logging.basicConfig(level=logging.INFO)

# Print script directory and its contents for debugging
logging.info(f"Notebook directory: {background_data_dir}")
logging.info(f"Directory contents: {os.listdir(background_data_dir)}")


# Define the service account file path
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE',os.path.join(background_data_dir, 'GoogleAuth.json'))

# Print the path being used
logging.info(f"Attempting to use service account file: {SERVICE_ACCOUNT_FILE}")


# Check if the file exists
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")


# GET DATA 

## REPORTING DATE

In [ ]:
def get_columns_and_types(directory: str) -> None:
    try:
        # Establish connection to SQLite database
        conn = sqlite3.connect(f"{directory}/master_data.db")
        cursor = conn.cursor()
        print("Successfully connected to the database")

        # Get all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        for table in tables:
            table_name = table[0]
            print(f"\nTable: {table_name}")

            # Get column information for the table
            cursor.execute(f"PRAGMA table_info('{table_name}')")
            columns = cursor.fetchall()

            # Print column names and their data types
            for column in columns:
                column_name = column[1]  # Name is in the second field
                data_type = column[2]   # Data type is in the third field
                print(f"  Column: {column_name}, Data Type: {data_type}")

    except sqlite3.Error as e:
        print(f"Error connecting to SQLite: {e}")

    finally:
        if 'conn' in locals():
            cursor.close()
            conn.close()
            print("\nSQLite connection closed")


In [ ]:
if __name__ == "__main__":
    get_columns_and_types(background_data_dir)

In [27]:
reporting_date = date(2025, 5, 2)
print(reporting_date)
reporting_date_formatted = reporting_date.strftime("%d %b %Y")
print(reporting_date_formatted)

2025-05-02
02 May 2025


In [31]:
datetime.today().strftime('%Y-%m-%d')

'2025-05-24'

In [ ]:
datetime.today()

reporting_date = date(2025, 5, 2)

reporting_date_formatted = reporting_date.strftime("%d %b %Y")

# CONNECT
conn = sqlite3.connect(f"{background_data_dir}/master_data.db")
cursor = conn.cursor()
## RETRIEVE ROW 
cursor.execute("SELECT * FROM MasterData WHERE ReportingDate = ?", (reporting_date_formatted,))
row = cursor.fetchone()


"""
adding YearWeek and altering the if then loop logic to account for the new column
DID NOT WORK since YearWeek is INTEGER and not a string

if row:
    column_names = [description[0] for description in cursor.description]
    for key, value in zip(column_names, row):
        if key in ["StartDate", "EndDate", "YearWeek", "ReportingDate"] and isinstance(value, str):
            try: 
                if key in ["StartDate", "EndDate"]:
                    value = datetime.strptime(value, "%Y-%m-%d").date() 
                elif key == "ReportingDate": 
                    # assuming ReportingDate is in 'dd MMM YYYY' format
                    value = datetime.strptime(value, "%d %b %Y").date()
                else: 
                    value = str(value)
            except ValueError:
                value = None 
                # store the value as a global variable (if needed)
            globals()[key] = value
            # print out the key, value, and its type
            print(f"{key} = {value} ({type(value).__name__})")
else:
    print(f"No row found for ReportingDate = {reporting_date_formatted}")
"""

if row:
    column_names = [description[0] for description in cursor.description]
    for key, value in zip(column_names, row):
        if key in ["StartDate", "EndDate", "ReportingDate"] and isinstance(value, str):
            try:
                if key in ["StartDate", "EndDate"]:
                    value = datetime.strptime(value, "%Y-%m-%d").date()
                elif key == "ReportingDate":
                    value = datetime.strptime(value, "%d %b %Y").date()
            except ValueError:
                print(f"Warning: Could not parse date for {key}: {value}")
                value = None
        elif key == "YearWeek":
            # Handle YearWeek as an integer (or convert if needed)
            value = int(value) if value is not None else None
        else:
            # Keep other columns as-is
            pass

        if value is not None or key in ["StartDate", "EndDate", "YearWeek", "ReportingDate"]:
                globals()[key] = value
                print(f"{key} = {value} ({type(value).__name__})")
else:
    print(f"No row found for ReportingDate = {reporting_date_formatted}")
  
# CLOSE CONNECTION
conn.close()

datetime.datetime(2025, 5, 24, 17, 17, 17, 863287)

In [ ]:
reporting_date = date(2025, 5, 2)

reporting_date_formatted = reporting_date.strftime("%d %b %Y")

# CONNECT
conn = sqlite3.connect(f"{background_data_dir}/master_data.db")
cursor = conn.cursor()
## RETRIEVE ROW 
cursor.execute("SELECT * FROM MasterData WHERE ReportingDate = ?", (reporting_date_formatted,))
row = cursor.fetchone()


"""
adding YearWeek and altering the if then loop logic to account for the new column
DID NOT WORK since YearWeek is INTEGER and not a string

if row:
    column_names = [description[0] for description in cursor.description]
    for key, value in zip(column_names, row):
        if key in ["StartDate", "EndDate", "YearWeek", "ReportingDate"] and isinstance(value, str):
            try: 
                if key in ["StartDate", "EndDate"]:
                    value = datetime.strptime(value, "%Y-%m-%d").date() 
                elif key == "ReportingDate": 
                    # assuming ReportingDate is in 'dd MMM YYYY' format
                    value = datetime.strptime(value, "%d %b %Y").date()
                else: 
                    value = str(value)
            except ValueError:
                value = None 
                # store the value as a global variable (if needed)
            globals()[key] = value
            # print out the key, value, and its type
            print(f"{key} = {value} ({type(value).__name__})")
else:
    print(f"No row found for ReportingDate = {reporting_date_formatted}")
"""

if row:
    column_names = [description[0] for description in cursor.description]
    for key, value in zip(column_names, row):
        if key in ["StartDate", "EndDate", "ReportingDate"] and isinstance(value, str):
            try:
                if key in ["StartDate", "EndDate"]:
                    value = datetime.strptime(value, "%Y-%m-%d").date()
                elif key == "ReportingDate":
                    value = datetime.strptime(value, "%d %b %Y").date()
            except ValueError:
                print(f"Warning: Could not parse date for {key}: {value}")
                value = None
        elif key == "YearWeek":
            # Handle YearWeek as an integer (or convert if needed)
            value = int(value) if value is not None else None
        else:
            # Keep other columns as-is
            pass

        if value is not None or key in ["StartDate", "EndDate", "YearWeek", "ReportingDate"]:
                globals()[key] = value
                print(f"{key} = {value} ({type(value).__name__})")
else:
    print(f"No row found for ReportingDate = {reporting_date_formatted}")
  
# CLOSE CONNECTION
conn.close()

## ADDITIONAL DATA ON GOOGLE DRIVE
NOTICE: 
Additional Data worksheet has been changed without keeping record of previous versions (ie. what this script was based on)

<strong> in future CRUCIAL to keep track of ALL alterations to script AND source</strong>

In [ ]:
def fetch_leave(reporting_date=reporting_date):
    reporting_date = pd.Timestamp(reporting_date)
    try:
        #gc, _ = authenticate_gdrive()
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Leave Sheet')
        data = pd.DataFrame(sheet.get_all_records())

        data['Leave - Start Date'] = pd.to_datetime(data['Leave - Start Date'], format="%d/%m/%Y", errors='coerce')
        data['Leave - End Date'] = pd.to_datetime(data['Leave - End Date'], format="%d/%m/%Y", errors='coerce')
        data['On Leave'] = False

        for index, row in data.iterrows():
            start = row['Leave - Start Date']
            end = row['Leave - End Date']

            if pd.notnull(start) and start < reporting_date <= end:
                data.at[index, 'On Leave'] = True

            elif pd.isnull(start) and pd.notnull(end) and end > reporting_date:
                data.at[index, 'On Leave'] = True

        return data

    except Exception as e:
        logging.error(f"Failed to fetch Leave Sheet: {e}")
        raise

In [ ]:
def clean_string(s):
    return str(s).strip().lower()

In [ ]:
def fetch_client_names():
    try:
    #gc, _ = authenticate_gdrive()
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Client Names Sheet')
        # Get all records from the sheet
        data = sheet.get_all_records()
        # Convert to DataFrame
        return pd.DataFrame(data)

    except Exception as e:
        logging.error(f"Failed to fetch Client Names Sheet: {e}")
        raise


In [ ]:
def fetch_kindle_employees():
    try:
        #gc, _ = authenticate_gdrive()
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open('Additional Data')
        sheet = spreadsheet.worksheet('Person Responsible Sheet')
        data = sheet.get_all_records()
        data = pd.DataFrame(data)
        return data

    except Exception as e:
        logging.error(f"Failed to fetch Person Responsible Sheet: {e}")
        raise

In [ ]:
def determine_person_responsible(reporting_date=reporting_date):
    reporting_date = pd.Timestamp(reporting_date)
    Employee_data = fetch_kindle_employees()
    LeaveStatus_data = fetch_leave(reporting_date)

    def assign_developer_turn(row):
        rotation_freq = row['Rotation Frequency (Weeks)']

        if pd.isna(rotation_freq) or rotation_freq == '':
        # Return Developer 1's name if valid, otherwise return empty string
            dev1 = row['Developer 1']
            if dev1 and dev1 != '?' and not pd.isna(dev1):
                return dev1.strip()
            
            return ''

        # Convert rotation_freq to an integer
        try:
            rotation_freq = int(rotation_freq)

        except (ValueError, TypeError):
            return ''

        if rotation_freq <= 0:
            return ''

        # Determine which developer columns to consider (Developer 1 to Developer n)
        developer_columns = ['Developer 1', 'Developer 2', 'Developer 3', 'Developer 4'][:rotation_freq]

        # Get the list of developers from the relevant columns
        developers = []
        for col in developer_columns:
            dev = row[col]
            if dev and dev != '?' and not pd.isna(dev):

                # Handle multiple developers in the same column (e.g., "Joel;Brendan")
                developers.extend([d.strip() for d in dev.split(';') if d.strip()])

        if not developers:
            return ''

        # Ensure current_week_number is an integer
        if not isinstance(WeekNumber, int):
            raise ValueError(f"current_week_number must be an integer, got {type(WeekNumber)}: {WeekNumber}")

        # Calculate the developer index
        developer_index = (WeekNumber % rotation_freq) % len(developers)

        # Verify developer_index is an integer
        if not isinstance(developer_index, int):
            raise ValueError(f"developer_index is not an integer, got {type(developer_index)}: {developer_index}")

        return developers[developer_index]

    Employee_data['Developer'] = Employee_data.apply(assign_developer_turn, axis=1)
    on_leave_list = LeaveStatus_data.loc[LeaveStatus_data['On Leave'] == True, 'Kindle Employee'].dropna().tolist()

    """
    SUGGESTED FOR LARGER DATASETS to use vectorized operations
    def determine_ba_responsible_df(df):
        conditions = [
            (df['Business Analyst 1'].notna() & ~df['Business Analyst 1'].isin(on_leave_list)),
            (df['Business Analyst 2'].notna() & ~df['Business Analyst 2'].isin(on_leave_list)),
            (df['In Case of Emergency'].notna())
        ]
        choices = [
            df['Business Analyst 1'],
            df['Business Analyst 2'],
            df['In Case of Emergency']
        ]
        return np.select(conditions, choices, default=None)
    """

    def determine_ba_responsible(row):
        ba1 = row['Business Analyst 1']
        ba2 = row['Business Analyst 2']
        emergency = row['In Case of Emergency']
        ba1_on_leave = pd.notna(ba1) and ba1 in on_leave_list
        ba2_on_leave = pd.notna(ba2) and ba2 in on_leave_list

        if pd.notna(ba1) and not ba1_on_leave:
            return ba1

        elif pd.notna(ba2) and not ba2_on_leave:
            return ba2

        elif pd.notna(emergency):
            return emergency

        else:
            return None

    Employee_data['BA Responsible'] = Employee_data.apply(determine_ba_responsible, axis=1)

    def determine_dev_responsible(row):

        dev = row['Developer']
        dev2 = row['Developer 2']
        dev3 = row['Developer 3']
        dev4 = row['Developer 4']
        emergency = row['In Case of Emergency']

        dev_on_leave = pd.notna(dev) and dev in on_leave_list
        dev2_on_leave = pd.notna(dev2) and dev2 in on_leave_list
        dev3_on_leave = pd.notna(dev3) and dev3 in on_leave_list
        dev4_on_leave = pd.notna(dev4) and dev4 in on_leave_list

        if pd.notna(dev) and not dev_on_leave:
            return dev

        elif pd.notna(dev2) and not dev2_on_leave:
            return dev2

        elif pd.notna(dev3) and not dev3_on_leave:
            return dev3

        elif pd.notna(dev4) and not dev4_on_leave:
            return dev4

        elif pd.notna(emergency):
            return emergency

        else:
            return None

    Employee_data['DEV Responsible'] = Employee_data.apply(determine_dev_responsible, axis=1)
    Employee_data = Employee_data.drop(['Business Analyst 1', 'Business Analyst 2', 'Key Resource','Developer 1', 'Developer 2', 'Developer 3', 'Developer 4','Rotation Frequency (Weeks)', 'In Case of Emergency', 'Developer'], axis=1)

    return Employee_data

In [ ]:
def fetch_current_errors():
    try:
        #gc, _ = authenticate_gdrive()
        creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
        gc = gspread.authorize(creds)
        spreadsheet = gc.open('ERROR REPORT Current Week')
        current_sheet = spreadsheet.worksheet(CurrentSheetName)
        data = current_sheet.get_all_records()
        return pd.DataFrame(data)

    except Exception as e:
        logging.error(f"Failed to fetch Sheet: {e}")
        raise

## PREVIOUS DATA from db

In [ ]:
def fetch_previous(YearWeek=YearWeek):
    db_path = os.path.join(background_data_dir, 'previous_errors_n_reponse_times_data.db')
    engine = create_engine(f'sqlite:///{db_path}')
    query = f"""
        SELECT * FROM PreviousErrorData
        WHERE CAST(SUBSTR(Period, 1, 6) AS INTEGER) < {YearWeek}
        """
    return pd.read_sql(query, engine)

## RESPONSE TIMES from local device

In [ ]:
def fetch_response_times(data_folder = data_supplied_dir):
    run_date = reporting_date - timedelta(days=1)
    date_string = f"{run_date.day:02d} {run_date.month:02d} {run_date.year}"
    target_filename = f"KIT 3 Online Reporting Data {date_string}.xlsx"

    files = os.listdir(data_folder)
    matched_file = None

    for file in files:
        if clean_string(file) == clean_string(target_filename):
            matched_file = file
            break

    if not matched_file:
        available_files = ", ".join(files)
        raise FileNotFoundError(f"File '{target_filename}' not found. Available files: {available_files}")


    file_path = os.path.join(data_folder, matched_file)


    try:
        xl = pd.ExcelFile(file_path)
    except Exception as e:
        raise Exception(f"File found but couldn't be accessed: {str(e)}")
  

    sheet_name = "This week response times"
    if sheet_name not in xl.sheet_names:
        sheet_name = xl.sheet_names[1]
        # Default to last sheet

    try:
        data = xl.parse(sheet_name, header=0)
    except Exception as e:
        raise Exception(f"No valid data found in the '{sheet_name}' sheet: {str(e)}")


    client_name_lookup = {}
    client_names_data = fetch_client_names()

    for _, row in client_names_data.iterrows():
        target = row['Client Names'].strip() if pd.notnull(row['Client Names']) else ''
        if pd.notnull(row['Other']) and row['Other'].lower() != 'null':
            aliases = [alias.strip() for alias in str(row['Other']).split(',')]
            
            for alias in aliases:
                client_name_lookup[clean_string(alias)] = target

        client_name_lookup[clean_string(target)] = target


    def standardize_company_name(ColumnName):
        if pd.isnull(ColumnName):
            return ''

        cleaned = clean_string(ColumnName)
        return client_name_lookup.get(cleaned, ColumnName)
        # fallback to original if not found

    data['Company'] = data['Company'].apply(standardize_company_name)
    desired_columns = ['Company', 'Average Response Time', 'Minimum Response Time','Maximum Response Time','Period', 'NewOnlineCount']
    existing_columns = [col for col in desired_columns if col in data.columns]
    data = data[existing_columns]

    # data.to_excel("ResponseTimesQuery.xlsx", index=False)

    return data

# DETERMINE CURRENT ERROR COUNTS

In [ ]:
def error_count():
    pd.set_option('future.no_silent_downcasting', True)
    data = fetch_current_errors()
    client_names_data = fetch_client_names()
    client_name_lookup = {}

    for _, row in client_names_data.iterrows():
        target = row['Client Names'].strip() if pd.notnull(row['Client Names']) else ''
        # Handle aliases in 'Other' column
        if pd.notnull(row['Other']):
            aliases = [alias.strip() for alias in str(row['Other']).split(',') if alias.strip()]
            for alias in aliases:
                client_name_lookup[clean_string(alias)] = target

        # Map the target name itself
        client_name_lookup[clean_string(target)] = target

    # Define function to standardize company names
    def standardize_company_name(name):
        if pd.isna(name):
            return np.nan
        
        cleaned = clean_string(name)
        return client_name_lookup.get(cleaned, name)
        # Fallback to original if not found

    # Apply standardization to 'Client' column
    data['Client'] = data['Client'].apply(standardize_company_name)
    data['Client'] = data['Client'].replace('', np.nan)
    data['Period'] = data['Period'].replace('', np.nan)
    data['Client'] = data['Client'].ffill().bfill()
    data['Period'] = data['Period'].ffill().bfill()
    data = data.infer_objects(copy=False)
    data['Functional Error Y/N'] = data['Functional Error Y/N'].astype(str).str.strip().str.upper()

    grouped = data.groupby(['Client', 'Period', 'Functional Error Y/N'], as_index=False)['No of times error occurred'].sum()
    grouped.rename(columns={'No of times error occurred': 'Total Errors'}, inplace=True)

    # Step 2: Rename column to remove spaces/slashes
    grouped.rename(columns={'Functional Error Y/N': 'FunctionalError_Y/N'}, inplace=True)

    # Step 3: Pivot Y/N values into columns
    pivoted = grouped.pivot_table(
        index=['Client', 'Period'],
        columns='FunctionalError_Y/N',
        values='Total Errors',
        aggfunc='sum',
        fill_value=0).reset_index()

    pivoted = pivoted.infer_objects(copy=False)
    # Flatten the column index created by the pivot (required!)
    pivoted.columns.name = None
    # Remove the "FunctionalError_Y/N" header
    pivoted = pivoted.drop(columns=[''])

    # Step 4: Rename pivoted columns for clarity
    pivoted.rename(columns={'Y': 'FunctionalErrors', 'N': 'NonFunctionalErrors'}, inplace=True)

    return pivoted

## JOIN WITH PREVIOUS AND RESPONSE TIMES

In [ ]:
def extract_dates(text):
    match = re.search(r'(\d{1,2} \w{3}) \d{4} - (\d{1,2} \w{3}) \d{4}', text)
    if match:
        return f"{match.group(1)} - {match.group(2)}"

    return None
    # or return text if no match is found

In [ ]:
def concat_errors():
    ResponseTimes_data = fetch_response_times()
    ResponseTimes_data = ResponseTimes_data[~ResponseTimes_data['Period'].isna()] #remove total row

    ErrorCount_data = error_count()
    ErrorCount_data = ErrorCount_data.rename(columns={'Client': 'Company'})
    ErrorCount_data = ErrorCount_data.drop(['Period'], axis=1) # can later be used as checK

    ErrorReport_data = pd.merge(ResponseTimes_data, ErrorCount_data, on=['Company'], how='left')
    columns_to_convert = ['FunctionalErrors', 'NonFunctionalErrors']
    for col in columns_to_convert:
        ErrorReport_data[col] = pd.to_numeric(ErrorReport_data[col], errors='coerce').astype('Int64')

    # Database path
    db_path = os.path.join(background_data_dir, 'previous_errors_n_reponse_times_data.db')

    # Create SQLAlchemy engine for SQLite
    engine = create_engine(f'sqlite:///{db_path}')

    # Verify the Period of new data against the last entry
    try:
        with engine.connect() as conn:
            # Get the last entry's Period from PreviousErrorData (most recent by Period)
            last_entry = pd.read_sql("SELECT Period FROM PreviousErrorData ORDER BY Period DESC LIMIT 1", conn)
            if last_entry.empty:
                print("Warning: PreviousErrorData table is empty. Cannot verify Period.")
            else:
                last_period = last_entry['Period'].iloc[0]
                last_period_prefix = int(last_period[:6])

                # Get the Period prefix from ErrorReport_data (assuming all rows have the same prefix)
                new_period_prefix = int(ErrorReport_data['Period'].iloc[0][:6])
                # First 6 characters

                # Verify that the new year is exactly one more than the last year
                if new_period_prefix == last_period_prefix + 1:
                    print(f"Verification successful: New Period ({new_period_prefix}) is one month after last Period ({last_period_prefix}).")
                else:
                    print(f"Verification failed: New Period ({new_period_prefix}) is not one more than last Period ({last_period_prefix}).")

    except Exception as e:
        print(f"Error during verification: {e}")

    # Check for duplicates and append only unique records
    try:
        with engine.connect() as conn:
            # Read existing records from PreviousErrorData to check for duplicates
            existing_records = pd.read_sql("SELECT Period, Company FROM PreviousErrorData", conn)
            # Create a merged DataFrame to identify new records
            # Convert relevant columns to string to ensure consistent comparison
            ErrorReport_data['Period'] = ErrorReport_data['Period'].astype(str)
            ErrorReport_data['Company'] = ErrorReport_data['Company'].astype(str)
            existing_records['Period'] = existing_records['Period'].astype(str)
            existing_records['Company'] = existing_records['Company'].astype(str)
            # Merge to find records in ErrorReport_data that don't exist in the database
            new_records = ErrorReport_data.merge(
                existing_records,
                on=['Period', 'Company'],
                how='left',
                indicator=True).query('_merge == "left_only"').drop(columns='_merge')

            if new_records.empty:
                print("No new records to append; all records already exist in PreviousErrorData.")
            else:
                # Append only the new records to the PreviousErrorData table
                new_records.to_sql('PreviousErrorData', con=engine, if_exists='append', index=False)

            print(f"Successfully appended {len(new_records)} new records to PreviousErrorData table.")

    except Exception as e:
        print(f"Error checking/appending data: {e}")


    PreviousErrors_data = fetch_previous()
    PreviousErrors_data = PreviousErrors_data.rename(columns={'AverageResponseTime': 'Average Response Time', 'MinimumResponseTime': 'Minimum Response Time', 'MaximumResponseTime':'Maximum Response Time'})

    ErrorReport_data = pd.concat([PreviousErrors_data, ErrorReport_data], ignore_index=True)

    return ErrorReport_data

# COPY DATA TO REPORTING WORKBOOK

In [ ]:
#def write_dataframe_to_sheet():
DataErrors_n_ResponseTimes_data = concat_errors()
DataErrors_n_ResponseTimes_data = DataErrors_n_ResponseTimes_data.where(pd.notnull(DataErrors_n_ResponseTimes_data), None)

try:
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(creds)
    spreadsheet = gc.open('KIT 3 Online Reporting Data')
    sheet = spreadsheet.worksheet('Data Errors and Reponse Times')
        # Directly convert NaN to None without fillna
        # data = [DataErrors_n_ResponseTimes_data.columns.tolist()] + DataErrors_n_ResponseTimes_data.where(pd.notnull(DataErrors_n_ResponseTimes_data), None).values.tolist()
        # Fill numeric columns with 0 and non-numeric with ''
    data_filled = DataErrors_n_ResponseTimes_data.copy()
    for col in data_filled.columns:
        if data_filled[col].dtype == 'Float64':
            data_filled[col] = data_filled[col].fillna(0)
        else:
            data_filled[col] = data_filled[col].fillna('')

    data = [data_filled.columns.tolist()] + data_filled.astype(object).where(pd.notnull(data_filled), None).values.tolist()
    execute_with_backoff(sheet.clear)
    execute_with_backoff(sheet.update, data, 'A1', value_input_option='RAW')

    logging.info(f"Overwrote {len(DataErrors_n_ResponseTimes_data)} rows in {sheet} in 'Data Errors and Reponse Times'")
    #return

except Exception as e:
    logging.error(f"Failed to write DataFrame to sheet: {e}")
    raise

#write_dataframe_to_sheet() 

In [ ]:
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '1m49V3mKZiuTatraSSGqo1xHLIhcQRmJZO90S4Y72vcc') # CURRENT
DEST_SPREADSHEET_ID = os.getenv('DEST_SPREADSHEET_ID', '1oc1zL7BdlRFFT68cZV46ctvHi5q-ZosaPk79HvireWI') # KIT 3 Online Reporting Data

# Validate service account file
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")

# Authenticate with Google Sheets
try:
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(creds)
    sheets_service = build('sheets', 'v4', credentials=creds)

except Exception as e:
    logging.error(f"Failed to authenticate with Google Sheets: {e}")
    raise

# Open source and destination spreadsheets
spreadsheet = execute_with_backoff(gc.open_by_key, SPREADSHEET_ID)
dest_spreadsheet = execute_with_backoff(gc.open_by_key, DEST_SPREADSHEET_ID)

# Cache worksheets to reduce API calls
source_worksheets = execute_with_backoff(spreadsheet.worksheets)
dest_worksheets = execute_with_backoff(dest_spreadsheet.worksheets)

source_worksheet_ids = {sheet.title: sheet.id for sheet in source_worksheets}
dest_worksheet_ids = {sheet.title: sheet.id for sheet in dest_worksheets}

# Define sheet names
source_sheet_name = CurrentSheetName
dest_sheet_name = 'Current Week Error'

# Copy and rename sheet if needed
if source_sheet_name in source_worksheet_ids and source_sheet_name not in dest_worksheet_ids:
    copy_response = execute_with_backoff(
        sheets_service.spreadsheets().sheets().copyTo,
        spreadsheetId=SPREADSHEET_ID,
        sheetId=source_worksheet_ids[source_sheet_name],
        body={'destinationSpreadsheetId': DEST_SPREADSHEET_ID})

    copied_sheet_id = copy_response['sheetId']
    logging.info(f"Copied sheet '{source_sheet_name}' to destination.")

    # Delete existing destination sheet if it exists
    if dest_sheet_name in dest_worksheet_ids:
        delete_request = {'requests': [{'deleteSheet': {'sheetId': dest_worksheet_ids[dest_sheet_name]}}]}
        
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=DEST_SPREADSHEET_ID,
            body=delete_request)

        logging.info(f"Deleted existing sheet '{dest_sheet_name}' from destination.")

    # Rename the copied sheet
    rename_request = {'requests': [{'updateSheetProperties': {'properties': {'sheetId': copied_sheet_id,'title': dest_sheet_name},'fields': 'title'}}]}

    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=DEST_SPREADSHEET_ID,
        body=rename_request)

    logging.info(f"Renamed copied sheet to '{dest_sheet_name}' in destination.")

else:
    logging.info(f"No sheet copied. Either '{source_sheet_name}' doesn't exist in source or '{dest_sheet_name}' already exists in destination.")

In [ ]:
SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '13IpNi1rFg8luMX8t_OlhCbAQcEhkZkZUmwFWP3C_8N8') # FOLLOW UP
DEST_SPREADSHEET_ID = os.getenv('DEST_SPREADSHEET_ID', '1oc1zL7BdlRFFT68cZV46ctvHi5q-ZosaPk79HvireWI')

# Validate service account file
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")

# Authenticate with Google Sheets
try:
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
    gc = gspread.authorize(creds)
    sheets_service = build('sheets', 'v4', credentials=creds)

except Exception as e:
    logging.error(f"Failed to authenticate with Google Sheets: {e}")
    raise

# Open source and destination spreadsheets
spreadsheet = execute_with_backoff(gc.open_by_key, SPREADSHEET_ID)
dest_spreadsheet = execute_with_backoff(gc.open_by_key, DEST_SPREADSHEET_ID)
 

# Cache worksheets to reduce API calls
source_worksheets = execute_with_backoff(spreadsheet.worksheets)
dest_worksheets = execute_with_backoff(dest_spreadsheet.worksheets)

source_worksheet_ids = {sheet.title: sheet.id for sheet in source_worksheets}
dest_worksheet_ids = {sheet.title: sheet.id for sheet in dest_worksheets}

# Define sheet names
source_sheet_name = PreviousSheetName
dest_sheet_name = 'Feedback on Previous Week Error'

# Copy and rename sheet if needed
if source_sheet_name in source_worksheet_ids and source_sheet_name not in dest_worksheet_ids:
    copy_response = execute_with_backoff(
        sheets_service.spreadsheets().sheets().copyTo,
        spreadsheetId=SPREADSHEET_ID,
        sheetId=source_worksheet_ids[source_sheet_name],
        body={'destinationSpreadsheetId': DEST_SPREADSHEET_ID})

    copied_sheet_id = copy_response['sheetId']
    logging.info(f"Copied sheet '{source_sheet_name}' to destination.")

    # Delete existing destination sheet if it exists
    if dest_sheet_name in dest_worksheet_ids:
        delete_request = {'requests': [{'deleteSheet': {'sheetId': dest_worksheet_ids[dest_sheet_name]}}]}

        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=DEST_SPREADSHEET_ID,
            body=delete_request)

        logging.info(f"Deleted existing sheet '{dest_sheet_name}' from destination.")
  

    # Rename the copied sheet
    rename_request = {'requests': [{'updateSheetProperties': {'properties': {'sheetId': copied_sheet_id,'title': dest_sheet_name},'fields': 'title'}}]}

    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=DEST_SPREADSHEET_ID,
        body=rename_request)

    logging.info(f"Renamed copied sheet to '{dest_sheet_name}' in destination.")

else:
    logging.info(f"No sheet copied. Either '{source_sheet_name}' doesn't exist in source or '{dest_sheet_name}' already exists in destination.")


# ADD and COPY SHEETS FOR NEXT WEEK

In [ ]:
def filter_hide_requests(worksheet_ids: Dict[str, int], visible_titles: List[str], spreadsheet_id: str) -> List[Dict[str, Any]]:
    try:
        metadata = execute_with_backoff(
            sheets_service.spreadsheets().get,
            spreadsheetId=spreadsheet_id,
            fields='sheets.properties')

        current_sheets = metadata['sheets']
        visible_sheet_ids = [
            s['properties']['sheetId']
            for s in current_sheets
            if not s['properties'].get('hidden', False)]

        hide_requests = []
        for title, sheet_id in worksheet_ids.items():
            if title not in visible_titles:
                hide_requests.append({"updateSheetProperties": {"properties": {"sheetId": sheet_id,"hidden": True},"fields": "hidden"}})

        if len(hide_requests) >= len(visible_sheet_ids):
            logging.warning(f"Prevented hiding all sheets in spreadsheet {spreadsheet_id}.")
            if hide_requests:
                hide_requests.pop()
        return hide_requests

    except Exception as e:
        logging.error(f"Failed to fetch or process current sheet visibility: {e}")
        raise

In [ ]:
logging.basicConfig(level=logging.INFO)

# Define the service account file path
SERVICE_ACCOUNT_FILE = os.getenv('SERVICE_ACCOUNT_FILE',os.path.join(background_data_dir, 'GoogleAuth.json'))

# Print the path being useD
logging.info(f"Attempting to use service account file: {SERVICE_ACCOUNT_FILE}")

# Check if the file exists
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")

logging.info("File found, proceeding with authentication...")
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Google Sheets setup
REDEFINED_SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
CURRENT_SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '1m49V3mKZiuTatraSSGqo1xHLIhcQRmJZO90S4Y72vcc') # CURRENT
FOLLOWUP_SPREADSHEET_ID = os.getenv('SPREADSHEET_ID', '13IpNi1rFg8luMX8t_OlhCbAQcEhkZkZUmwFWP3C_8N8') # FOLLOW UP
TEMPLATE_NAME = 'TEMPLATE - Do not edit'

# Validate service account file
if not os.path.exists(SERVICE_ACCOUNT_FILE):
    logging.error(f"Service account file not found: {SERVICE_ACCOUNT_FILE}")
    raise FileNotFoundError(f"Service account file not found: {SERVICE_ACCOUNT_FILE}") 

# Authenticate with Google Sheets
try:
    creds = Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=REDEFINED_SCOPES)
    gc = gspread.authorize(creds)
    sheets_service = build('sheets', 'v4', credentials=creds)

except Exception as e:
    logging.error(f"Failed to authenticate with Google Sheets: {e}")
    raise


# Open spreadsheets
current_spreadsheet = execute_with_backoff(gc.open_by_key, CURRENT_SPREADSHEET_ID)
template_sheet = current_spreadsheet.get_worksheet_by_id(552319826)
followup_spreadsheet = execute_with_backoff(gc.open_by_key, FOLLOWUP_SPREADSHEET_ID)

# Cache worksheets to reduce API calls
current_worksheets = execute_with_backoff(current_spreadsheet.worksheets)
followup_worksheets = execute_with_backoff(followup_spreadsheet.worksheets)

source_worksheet_ids = {sheet.title: sheet.id for sheet in current_worksheets}
dest_worksheet_ids = {sheet.title: sheet.id for sheet in followup_worksheets}


# Define sheet names (replace with actual values or logic)
next_sheet_name = NextSheetName
followup_sheet_name = CurrentSheetName

# Create or open new worksheet
if next_sheet_name in source_worksheet_ids:
    logging.info(f"Worksheet '{next_sheet_name}' already exists. Deleting it.")
    sheet_id = source_worksheet_ids[next_sheet_name]
    delete_request = {'requests': [{'deleteSheet': {'sheetId': sheet_id}}]}

    try:
        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            body=delete_request)

        # Remove from cache
        del source_worksheet_ids[next_sheet_name]
        # Brief delay to ensure deletion is processed

        time.sleep(1)
        # Refresh worksheet list to confirm deletion
        current_worksheets = execute_with_backoff(current_spreadsheet.worksheets)
        source_worksheet_ids = {sheet.title: sheet.id for sheet in current_worksheets}
        if next_sheet_name in source_worksheet_ids:
            raise Exception(f"Failed to delete worksheet '{next_sheet_name}'.")
        logging.info(f"Deleted worksheet '{next_sheet_name}'.")

    except Exception as e:
        logging.error(f"Error deleting worksheet '{next_sheet_name}': {e}")
        raise

  
# Create new sheet
try:
    new_sheet = execute_with_backoff(
        current_spreadsheet.duplicate_sheet,
        source_sheet_id=template_sheet.id,
        new_sheet_name=next_sheet_name)
    
    source_worksheet_ids[next_sheet_name] = new_sheet.id
    logging.info(f"Created worksheet '{next_sheet_name}'.")

except Exception as e:
    logging.error(f"Error creating worksheet '{next_sheet_name}': {e}")
    raise

# Update 'Period' column
values = execute_with_backoff(new_sheet.get_all_values)
header = values[0]

try:
    period_col_index = header.index("Period")
    period_col_letter = chr(ord('A') + period_col_index)

except ValueError:
    raise Exception("Column 'Period' not found.")

updates = [{'range': f"{period_col_letter}{row_idx}", 'values': [[next_sheet_name]]} 
           for row_idx, row in enumerate(values[1:], start=2) 
           if len(row) > period_col_index and row[period_col_index].strip()]

if updates:
    execute_with_backoff(new_sheet.batch_update, updates)
    logging.info(f"Updated {len(updates)} 'Period' cells.")

# Copy followup sheet to destination spreadsheet if needed
if followup_sheet_name in source_worksheet_ids and followup_sheet_name not in dest_worksheet_ids:
    try:
        copy_response = execute_with_backoff(
            sheets_service.spreadsheets().sheets().copyTo,
            spreadsheetId=CURRENT_SPREADSHEET_ID,
            sheetId=source_worksheet_ids[followup_sheet_name],
            body={'destinationSpreadsheetId': FOLLOWUP_SPREADSHEET_ID})

        copied_sheet_id = copy_response['sheetId']
        rename_request = {'requests': [{'updateSheetProperties': {'properties': {'sheetId': copied_sheet_id,'title': followup_sheet_name},'fields': 'title'}}]}

        execute_with_backoff(
            sheets_service.spreadsheets().batchUpdate,
            spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
            body=rename_request)

        dest_worksheet_ids[followup_sheet_name] = copied_sheet_id
        logging.info(f"Copied and renamed sheet '{followup_sheet_name}' to destination.")

    except Exception as e:
        logging.error(f"Error copying sheet '{followup_sheet_name}': {e}")
        raise


#### HttpError                                 
Traceback (most recent call last)
Cell In[65], line 63
     60 delete_request = {'requests': [{'deleteSheet': {'sheetId': sheet_id}}]}
     62 try:
---> 63     execute_with_backoff(
     64         sheets_service.spreadsheets().batchUpdate,
     65         spreadsheetId=CURRENT_SPREADSHEET_ID,
     66         body=delete_request)
     68     # Remove from cache
     69     del source_worksheet_ids[next_sheet_name]

Cell In[13], line 6, in execute_with_backoff(func, max_retries, *args, **kwargs)
      4     result=func(*args,**kwargs)
      5     if isinstance(result,HttpRequest):
----> 6         result=result.execute()
      8     return result
     10 except(HttpError,gspread.exceptions.APIError) as e:

File c:\Users\nicola\Desktop\VisualCode Workspace\.venv\Lib\site-packages\googleapiclient\_helpers.py:130, in positional.<locals>.positional_decorator.<locals>.positional_wrapper(*args, **kwargs)
    128     elif positional_parameters_enforcement == POSITIONAL_WARNING:
    129         logger.warning(message)
--> 130 return wrapped(*args, **kwargs)

File c:\Users\nicola\Desktop\VisualCode Workspace\.venv\Lib\site-packages\googleapiclient\http.py:938, in HttpRequest.execute(self, http, num_retries)
    936     callback(resp)
    937 if resp.status >= 300:
--> 938     raise HttpError(resp, content, uri=self.uri)
    939 return self.postproc(resp, content)

HttpError: <HttpError 400 when requesting https://sheets.googleapis.com/v4/spreadsheets/1m49V3mKZiuTatraSSGqo1xHLIhcQRmJZO90S4Y72vcc:batchUpdate?alt=json returned "Invalid requests[0].deleteSheet: You can't remove all the visible sheets in a document.". Details: "Invalid requests[0].deleteSheet: You can't remove all the visible sheets in a document.">

In [ ]:
current_visible_sheets = [next_sheet_name]
followup_visible_sheets = [followup_sheet_name]

source_requests = filter_hide_requests(source_worksheet_ids, current_visible_sheets, CURRENT_SPREADSHEET_ID)
dest_requests = filter_hide_requests(dest_worksheet_ids, followup_visible_sheets, FOLLOWUP_SPREADSHEET_ID)

if source_requests:
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=CURRENT_SPREADSHEET_ID,
        body={"requests": source_requests})

    logging.info(f"Hid {len(source_requests)} sheets in source spreadsheet.")

if dest_requests:
    execute_with_backoff(
        sheets_service.spreadsheets().batchUpdate,
        spreadsheetId=FOLLOWUP_SPREADSHEET_ID,
        body={"requests": dest_requests})

    logging.info(f"Hid {len(dest_requests)} sheets in destination spreadsheet.")